In [3]:
import ctypes

# Define necessary structures
class XINPUT_VIBRATION(ctypes.Structure):
    _fields_ = [("wLeftMotorSpeed", ctypes.c_ushort),
                ("wRightMotorSpeed", ctypes.c_ushort)]

xinput = ctypes.windll.xinput1_1  # Load Xinput.dll

# Set up function argument types and return type
XInputSetState = xinput.XInputSetState
XInputSetState.argtypes = [ctypes.c_uint, ctypes.POINTER(XINPUT_VIBRATION)]
XInputSetState.restype = ctypes.c_uint

# Now we're ready to call it.  Set left motor to 100%, right motor to 50%
# for controller 0
vibration = XINPUT_VIBRATION(65535, 32768)
XInputSetState(0, ctypes.byref(vibration))

# You can also create a helper function like this:
def set_vibration(controller, left_motor, right_motor):
    vibration = XINPUT_VIBRATION(int(left_motor * 65535), int(right_motor * 65535))
    XInputSetState(controller, ctypes.byref(vibration))

# ... and use it like so
set_vibration(0, 0, 0.1)

In [22]:
import ctypes
from time import sleep

# Define necessary structures
class XINPUT_GAMEPAD(ctypes.Structure):
    _fields_ = [("wButtons", ctypes.c_ushort),
                ("bLeftTrigger", ctypes.c_ubyte),
                ("bRightTrigger", ctypes.c_ubyte),
                ("sThumbLX", ctypes.c_short),
                ("sThumbLY", ctypes.c_short),
                ("sThumbRX", ctypes.c_short),
                ("sThumbRY", ctypes.c_short)]

class XINPUT_STATE(ctypes.Structure):
    _fields_ = [("dwPacketNumber", ctypes.c_uint),
                ("Gamepad", XINPUT_GAMEPAD)]

xinput = ctypes.windll.xinput1_1  # Load Xinput.dll

# Set up function argument types and return type
XInputGetState = xinput.XInputGetState
XInputGetState.argtypes = [ctypes.c_uint, ctypes.POINTER(XINPUT_STATE)]
XInputGetState.restype = ctypes.c_uint

# Function to get the trigger values (0 to 255)
def get_trigger_values(controller):
    state = XINPUT_STATE()
    XInputGetState(controller, ctypes.byref(state))
    return state.Gamepad.bLeftTrigger, state.Gamepad.bRightTrigger

# You can now use this function to monitor the trigger values.
while True:
    left_trigger, right_trigger = get_trigger_values(0)
    
    if left_trigger > 200:
        print("L2 trigger hard push")
    elif left_trigger > 50:
        print("L2 trigger soft push")
    
    if right_trigger > 200:
        print("R2 trigger hard push")
    elif right_trigger > 50:
        print("R2 trigger soft push")
    
    sleep(0.1)  # Adjust the sleep duration as needed


R2 trigger soft push
R2 trigger soft push
R2 trigger soft push
R2 trigger soft push
R2 trigger soft push
R2 trigger soft push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger soft push
R2 trigger soft push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger hard push
R2 trigger soft push
L2 trigger soft push
L2 trigger soft push
L2 trigger soft push
L2 trigger soft push
L2 trigger soft push
L2 trigger soft push
L2 trigger soft push
L2 trigger hard push
L2 trigger soft push
L2 trigger hard push
L2 trigger hard push
L2 trigger soft push
R2 trigger hard push
L2 trigger hard push
R2 trigger hard push
L2 trigger hard push
R2 trigger hard push
L2 trigger hard push
L2 trigger hard push


KeyboardInterrupt: 

In [41]:
import pygame
import math
import random
import ctypes

# Initialize Pygame
pygame.init()

# Constants
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 800
RED = (255, 0, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Rectangular strip properties
strip_height = WINDOW_HEIGHT // 3
strip_top = (WINDOW_HEIGHT - strip_height) // 2
strip_bottom = strip_top + strip_height

# Create the game window
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Haptic Simulation")

# Dot properties
dot_radius = 20
white_square_size = 40
red_dot_x = WINDOW_WIDTH // 2 - 5
red_dot_y = WINDOW_HEIGHT // 2 - 275
white_square_x = WINDOW_WIDTH // 4
white_square_y = WINDOW_HEIGHT // 4
blue_dot_x = random.randint(dot_radius, WINDOW_WIDTH - dot_radius)
blue_dot_y = random.randint(strip_top + dot_radius, strip_bottom - dot_radius)
blue_dot_speed = 0.8
blue_dot_speed_x = 0.8
blue_dot_speed_y = 0.8
blue_dot_speed_increment = 0.2
max_blue_dot_speed = 3.0
min_blue_dot_speed = 0.1
sensitivity_factor = 5

# Game loop
running = True
game_over = False
restart_pressed = False

# Initialize Pygame joystick module
pygame.joystick.init()

# Check if there is any joystick connected
joystick_count = pygame.joystick.get_count()
if joystick_count == 0:
    print("No joystick found. Exiting.")
    pygame.quit()
    exit()
else:
    joystick_available = True

# Get the first joystick
joystick = pygame.joystick.Joystick(0)
joystick.init()

# Define necessary structures for vibration
class XINPUT_VIBRATION(ctypes.Structure):
    _fields_ = [("wLeftMotorSpeed", ctypes.c_ushort),
                ("wRightMotorSpeed", ctypes.c_ushort)]

xinput = ctypes.windll.xinput1_1  # Load Xinput.dll

# Set up function argument types and return type
XInputSetState = xinput.XInputSetState
XInputSetState.argtypes = [ctypes.c_uint, ctypes.POINTER(XINPUT_VIBRATION)]
XInputSetState.restype = ctypes.c_uint

# Function to set vibration
def set_vibration(controller, left_motor, right_motor):
    vibration = XINPUT_VIBRATION(int(left_motor * 65535), int(right_motor * 65535))
    XInputSetState(controller, ctypes.byref(vibration))

font = pygame.font.Font(None, 36)  # Increase font size for restart text
restart_font = pygame.font.Font(None, 36)

accumulated_x = 0
accumulated_y = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.JOYAXISMOTION:
            if event.axis == 0:
                accumulated_x += event.value
            elif event.axis == 1:
                accumulated_y += event.value
        elif event.type == pygame.JOYBUTTONDOWN:
            if event.button == 3:
                blue_dot_speed += blue_dot_speed_increment
                blue_dot_speed = min(blue_dot_speed, max_blue_dot_speed)
                speed_magnitude = math.sqrt(blue_dot_speed_x**2 + blue_dot_speed_y**2)
                blue_dot_speed_x = (blue_dot_speed_x / speed_magnitude) * blue_dot_speed
                blue_dot_speed_y = (blue_dot_speed_y / speed_magnitude) * blue_dot_speed
            elif event.button == 1:
                blue_dot_speed -= blue_dot_speed_increment
                blue_dot_speed = max(blue_dot_speed, min_blue_dot_speed)
                speed_magnitude = math.sqrt(blue_dot_speed_x**2 + blue_dot_speed_y**2)
                blue_dot_speed_x = (blue_dot_speed_x / speed_magnitude) * blue_dot_speed
                blue_dot_speed_y = (blue_dot_speed_y / speed_magnitude) * blue_dot_speed

    red_dot_x += int(sensitivity_factor * accumulated_x)
    red_dot_y += int(sensitivity_factor * accumulated_y)

    accumulated_x = 0
    accumulated_y = 0

    if game_over:
        set_vibration(0, 0, 0)
        restart_text = font.render("Press 'A' to Restart", True, WHITE)
        restart_text_rect = restart_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
        screen.blit(restart_text, restart_text_rect.topleft)

        # Check if the 'A' button is pressed to restart the game
        a_button_pressed = pygame.joystick.Joystick(0).get_button(0)
        if a_button_pressed and not restart_pressed:
            # Reset game state
            red_dot_x = WINDOW_WIDTH // 2 - 5
            red_dot_y = WINDOW_HEIGHT // 2 - 275
            white_square_x = WINDOW_WIDTH // 4
            white_square_y = WINDOW_HEIGHT // 4
            blue_dot_x = random.randint(dot_radius, WINDOW_WIDTH - dot_radius)
            blue_dot_y = random.randint(strip_top + dot_radius, strip_bottom - dot_radius)
            game_over = False
            restart_pressed = True
        elif not a_button_pressed:
            restart_pressed = False

    distance = math.sqrt((red_dot_x - blue_dot_x)**2 + (red_dot_y - blue_dot_y)**2)
    if distance < dot_radius * 2:
        game_over = True

    if strip_top < red_dot_y < strip_bottom:
        x_distance = red_dot_x - blue_dot_x
        intensity = max(0, min(1, 1 - abs(x_distance) / WINDOW_WIDTH))
        if x_distance > 0:
            set_vibration(0, intensity, 0)
        elif x_distance < 0:
            set_vibration(0, 0, intensity)
        else:
            set_vibration(0, 0, 0)
    else:
        set_vibration(0, 0, 0)

    blue_dot_x += blue_dot_speed_x
    blue_dot_y += blue_dot_speed_y

    if blue_dot_x > WINDOW_WIDTH - dot_radius or blue_dot_x < dot_radius:
        blue_dot_speed_x = -blue_dot_speed_x

    if blue_dot_y > strip_bottom - dot_radius or blue_dot_y < strip_top + dot_radius:
        blue_dot_speed_y = -blue_dot_speed_y

    blue_dot_x = max(min(blue_dot_x, WINDOW_WIDTH - dot_radius), dot_radius)
    blue_dot_y = max(min(blue_dot_y, strip_bottom - dot_radius), strip_top + dot_radius)

    screen.fill(BLACK)
    pygame.draw.rect(screen, WHITE, (0, strip_top, WINDOW_WIDTH, strip_height))
    pygame.draw.rect(screen, WHITE, (white_square_x - white_square_size // 2 + 175, white_square_y - white_square_size // 2 - 100, white_square_size * 2, white_square_size * 2))
    pygame.draw.circle(screen, RED, (red_dot_x, red_dot_y), dot_radius // 1.5)
    pygame.draw.circle(screen, BLUE, (blue_dot_x, blue_dot_y), dot_radius)

    objective_text = font.render("Catch the Blue Ball!", True, WHITE)
    screen.blit(objective_text, (275, 20))

    increase_difficulty_text = font.render("Press Triangle to increase difficulty!", True, WHITE)
    screen.blit(increase_difficulty_text, (10, 550))

    decrease_difficulty_text = font.render("Press Circle to decrease difficulty!", True, WHITE)
    screen.blit(decrease_difficulty_text, (10, 650))

    pygame.display.flip()

pygame.quit()
